In [4]:
import os
import pandas as pd
import gmaps
import googlemaps as g
import yaml
import numpy
import matplotlib.pyplot as plt
from IPython.display import Image
import foursquare
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split
from sklearn import linear_model


%matplotlib inline
plt.style.use('ggplot')

pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 20)
pd.set_option('display.notebook_repr_html', True)

# Loading dataset
df = pd.read_csv(os.path.join('..', 'datasets', 'venue_attributes.csv'))
# df = df.set_index(id)
df.drop(['Unnamed: 0'],inplace = True, axis = 1)
df.head()


,foursquare_id,foursquare_name,inspection_score,checkinsCount,createdAt,hasMenu,isVerified,photos,price,rating,ratingSignals,tipCount
0,3fd66200f964a52001ed1ee3,Pacific Cafe,86.000000,1389,1071014400,True,True,78,2.0,8.1,59.0,24
1,3fd66200f964a52005ed1ee3,Taiwan Restaurant 台灣飯店,74.111111,2173,1071014400,True,True,185,1.0,7.2,37.0,12
2,3fd66200f964a52006ed1ee3,Tanuki Restaurant 狸壽司,92.000000,1149,1071014400,True,True,46,2.0,7.9,34.0,13
3,3fd66200f964a52013ed1ee3,Acquerello,98.666667,1185,1071014400,True,True,72,4.0,8.5,108.0,41
4,3fd66200f964a52017ed1ee3,Ebisu,80.000000,5047,1071014400,True,True,254,3.0,9.4,307.0,88


In [5]:
df.createdAt.describe()

count    4.482000e+03
mean     1.287802e+09
std      9.146457e+07
min      1.071014e+09
25%      1.247093e+09
50%      1.267637e+09
75%      1.344520e+09
max      1.503523e+09
Name: createdAt, dtype: float64